<a href="https://colab.research.google.com/github/jai2shan/TSAI-EVA40-Assignments/blob/master/Assignment%20S6%20-%20Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.listdir()

['.config', 'asgnmnt6-0.0.32-py3-none-any.whl', 'sample_data']

In [2]:
pip install asgnmnt6-0.0.32-py3-none-any.whl

Processing ./asgnmnt6-0.0.32-py3-none-any.whl


## Loading Packages

In [0]:
from __future__ import print_function
import torch
import torch.optim as optim


import os

from asgnmnt6.GBNModel import GBNNet
from asgnmnt6.Model import Net
from asgnmnt6.DataTransformation import DataTransformation
from asgnmnt6.TrainTest import TrainTest

## Loading Train and Test Data

In [5]:
cuda = torch.cuda.is_available()
print("CUDA Available?", torch.cuda.is_available())

dt = DataTransformation()
# train dataloader
train_loader = dt.TrainPrep()
# test dataloader
test_loader = dt.TrainPrep()

CUDA Available? False


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


## Model Sumary

In [6]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
print("####### with Ghost Batch Normalization######")
model = GBNNet().to(device)
summary(model, input_size=(1, 28, 28))


cpu
####### with Ghost Batch Normalization######
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
    GhostBatchNorm-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
    GhostBatchNorm-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9            [-1, 8, 24, 24]             128
        MaxPool2d-10            [-1, 8, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
   GhostBatchNorm-13           [-1, 16, 10, 10]       

In [7]:
print("####### with Batch Normalization######")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

####### with Batch Normalization######
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9            [-1, 8, 24, 24]             128
        MaxPool2d-10            [-1, 8, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32


In [9]:
from torch.optim.lr_scheduler import StepLR

models_ = {
            1:{'L1' : False,'L2' : False,'GBN':False},
            2:{'L1' : False,'L2' : False,'GBN':True},
            3:{'L1' : True,'L2' : False,'GBN':False},
            4:{'L1' : True,'L2': False,'GBN':True},
            5:{'L1' : False,'L2' : True,'GBN':False},
            6:{'L1' : False,'L2' : True,'GBN':True},
            7:{'L1' : True,'L2' : True,'GBN':False},
            8:{'L1' : True,'L2': False,'GBN':True}
            }

for i in list(range(1,9)):
    print('##############################################################')
    print('L1:{},L2:{},GBN:{}'.format(models_[i]['L1'],models_[i]['L2'],models_[i]['GBN']))
    if (models_[i]['GBN']==False):
        model =  Net().to(device)
    else:
        model =  GBNNet().to(device)
    
    def Optimizer_(L2 = models_[i]['L2']):
        if (L2 == False):
            optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
        else:
            optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9,weight_decay = 0.00005)
        return optimizer
    optimizer = Optimizer_(L2 = True)
    scheduler = StepLR(optimizer, step_size=6, gamma=0.1)
    tt = TrainTest()
    EPOCHS = 2
    test_acc = 0
    for epoch in range(EPOCHS):
        print("EPOCH:", epoch)
        tt.train_(model, device, train_loader, optimizer, epoch,L1 = models_[i]['L1'])
        scheduler.step()
        acc = tt.test_(model, device, test_loader)
        if acc>=test_acc:
            models_[i]['Saved Model'] = model.state_dict()
                
    models_[i]['Training Loss'] = tt.train_losses
    models_[i]["Training Accuracy"] = tt.train_acc[4000:]
    models_[i]['Test Loss'] = tt.test_losses
    models_[i]["Test Accuracy"] = tt.test_acc

Loss=2.3039495944976807 Batch_id=1 Accuracy=7.03:   0%|          | 2/938 [00:00<01:19, 11.71it/s]

EPOCH: 0


Loss=0.05938779190182686 Batch_id=937 Accuracy=93.89: 100%|██████████| 938/938 [01:12<00:00, 12.93it/s]
Loss=0.017897598445415497 Batch_id=1 Accuracy=98.44:   0%|          | 2/938 [00:00<01:10, 13.23it/s]


Test set: Average loss: 0.0596, Accuracy: 58885/60000 (98.14%)

EPOCH: 1


Loss=0.056757569313049316 Batch_id=186 Accuracy=97.73:  20%|█▉        | 186/938 [00:14<00:57, 13.09it/s]

KeyboardInterrupt: ignored